<center><img src='https://raw.githubusercontent.com/Jangrae/img/master/ml_python.png' width=600/></center>

# 실습 내용

- Grid Search로 Decision Tree 알고리즘 모델을 튜닝합니다.


# 1.환경 준비

- 기본 라이브러리와 대상 데이터를 가져와 이후 과정을 준비합니다.

In [36]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [37]:
# 데이터 읽어오기
path = 'https://raw.githubusercontent.com/jangrae/csv/master/titanic.csv'
data = pd.read_csv(path)

# 2.데이터 이해

- 분석할 데이터를 충분히 이해할 수 있도록 다양한 탐색 과정을 수행합니다.

In [38]:
# 상위 몇 개 행 확인
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [39]:
# 변수 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [40]:
# 기초통계량 확인
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [41]:
# NaN 값 확인
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

# 3.데이터 준비

- 전처리 과정을 통해 머신러닝 알고리즘에 사용할 수 있는 형태의 데이터를 준비합니다.

**1) Title 변수 추가**

In [42]:
# 호칭 추출
data['Title'] = data['Name'].str.extract('([A-Za-z]+)\.', expand=False)

In [43]:
# 대표 호칭 이외는 Others로 변경
main_tit_list = ['Mr', 'Miss', 'Mrs']
data.loc[data['Title'].isin(main_tit_list)==False, ['Title']] = 'Others'

**2) 결측치 처리**

In [44]:
# Age 중앙값으로 채우기
data['Age'].fillna(data.groupby('Title')['Age'].transform('median'), inplace=True)

In [45]:
# Embarked 최빈값으로 채우기
data['Embarked'] = data['Embarked'].fillna('S')

**3) AgeGroup 변수 추가**

In [46]:
# 변수 추가
bins = [0, 16, 25, 35, 60, 100]
data['AgeGroup'] = pd.cut(data['Age'], 
                          bins=bins, 
                          labels=['0_16', '17_25', '26_35', '36_60', '61_100'])

**4) 불필요한 변수 제거**

In [47]:
# 불필요한 열 제거
drop_cols = ['PassengerId', 'Name', 'Age', 'Ticket', 'Cabin']
data.drop(drop_cols, axis=1, inplace=True)

**5) 가변수화**

In [48]:
# 가변수화
dummy_vars = ['Title', 'AgeGroup', 'Pclass', 'Sex', 'Embarked']

for v in dummy_vars:
    dummies = pd.get_dummies(data[v], prefix=v, drop_first=True)
    data = pd.concat([data, dummies], axis=1)
    data.drop(v, axis=1, inplace=True)

**6) x, y 분리**

In [49]:
# target 확인
target = 'Survived'

# 데이터 분리
x = data.drop(target, axis = 1)
y = data[target] 

**7) 학습용, 평가용 데이터 분리**

In [50]:
# 모듈 불러오기
from sklearn.model_selection import train_test_split

# 7:3으로 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=2022)

# 4.모델링

- 본격적으로 모델을 선언하고 학습하고 평가하는 과정을 진행합니다.
- 모델 선언시 max_depth=5, random_state=2022를 지정합니다.

In [51]:
# 불러오기
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report

In [52]:
# 선언하기
model = DecisionTreeClassifier(max_depth=5, random_state=2022)

In [53]:
# 학습하기
model.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=5, random_state=2022)

In [54]:
# 예측하기
y_pred = model.predict(x_test)

In [55]:
# 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[156  12]
 [ 36  64]]
              precision    recall  f1-score   support

           0       0.81      0.93      0.87       168
           1       0.84      0.64      0.73       100

    accuracy                           0.82       268
   macro avg       0.83      0.78      0.80       268
weighted avg       0.82      0.82      0.81       268



# 5.성능 튜닝

- Grid Search로 튜닝을 진행합니다.

**1) 모델링 & 튜닝**

In [56]:
# 파라미터 선언
  # max_depth: range(3, 21)
grid_param = {'max_depth':range(3, 21)}

# 확인
grid_param

{'max_depth': range(3, 21)}

In [57]:
# 불러오기
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [58]:
# 선언하기
model_dst = DecisionTreeClassifier(random_state=2022)

# Grid Search 모델 선언하기
  # cv=10
  # scoring='accuracy'
model = GridSearchCV(model_dst,         # 기본 모델 이름
                    grid_param,         # 앞에서 선언한 튜닝용 파라미터 변수
                    cv=10,              # K-Fold Cross Validation(default=5)
                    scoring='accuracy', # 평가 방법
                    n_jobs=-1)          # cpu성능 좋아지게 병렬처리

In [59]:
# 학습하기
model.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=2022),
             n_jobs=-1, param_grid={'max_depth': range(3, 21)},
             scoring='accuracy')

**2) 결과 확인**

In [60]:
model.cv_results_ 

{'mean_fit_time': array([0.00834973, 0.00812311, 0.00718043, 0.00735083, 0.00766063,
        0.00788424, 0.00740285, 0.00697486, 0.0097863 , 0.00735471,
        0.00802741, 0.00761149, 0.00800252, 0.00888913, 0.00882616,
        0.00857446, 0.00897868, 0.0086478 ]),
 'std_fit_time': array([0.00107548, 0.00130049, 0.00040999, 0.00030878, 0.00045717,
        0.00173969, 0.00056549, 0.00088821, 0.00679242, 0.00085836,
        0.0010465 , 0.00075181, 0.00191698, 0.00195957, 0.00287833,
        0.00188475, 0.00282418, 0.00286564]),
 'mean_score_time': array([0.0046845 , 0.00521343, 0.00496378, 0.00404062, 0.00396914,
        0.00405686, 0.00494013, 0.00370893, 0.00383329, 0.00489643,
        0.00526049, 0.00440686, 0.00461111, 0.00606387, 0.00414426,
        0.00460062, 0.00439072, 0.00432904]),
 'std_score_time': array([0.00088169, 0.00137002, 0.00256332, 0.00025451, 0.00059305,
        0.00040016, 0.00144597, 0.00064078, 0.00043727, 0.0021772 ,
        0.00411219, 0.00118296, 0.0007831 , 

In [61]:
# mean_test_score 확인
model.cv_results_['mean_test_score']

array([0.80593958, 0.81395289, 0.82352791, 0.81390169, 0.81395289,
       0.79953917, 0.80437788, 0.79633897, 0.77063492, 0.78026114,
       0.77862263, 0.77703533, 0.77706093, 0.77706093, 0.77544803,
       0.77867384, 0.78028674, 0.78028674])

In [62]:
# 최적의 파라미터
model.best_params_

{'max_depth': 5}

In [63]:
# 최고의 성능
model.best_score_

0.8235279057859701

In [64]:
# 최적의 성능을 보이는 모델 확인
model.best_estimator_.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 5,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 2022,
 'splitter': 'best'}

**3) 예측과 평가**

- 학습을 통해 예상한 성능과 실제 평가에 따른 성능은 차이가 있을 수 있습니다.
- 예선전에서 성적이 좋았다고 본선에서도 성적이 좋다고 보장할 수 없는 없겠지요?

In [65]:
# 예측하기
y_pred = model.predict(x_test)

In [66]:
# 평가하기
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[156  12]
 [ 36  64]]
              precision    recall  f1-score   support

           0       0.81      0.93      0.87       168
           1       0.84      0.64      0.73       100

    accuracy                           0.82       268
   macro avg       0.83      0.78      0.80       268
weighted avg       0.82      0.82      0.81       268

